In [1]:
%matplotlib notebook 

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.colors as colors

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import sunpy.map
import sunpy.data.sample
from sunpy.net import Fido, attrs as a
from sunpy.coordinates import Helioprojective, RotatedSunFrame, transform_with_sun_center
from sunpy.io import write_file
from sunpy.time import parse_time

import astropy.units as u
from astropy.coordinates import SkyCoord, ICRS
from astropy.wcs import WCS
from astropy.wcs.utils import skycoord_to_pixel
from astropy.utils.data import get_pkg_data_filename
from astropy.io import fits

from reproject import reproject_interp
from aiapy.calibrate import register, update_pointing
from aiapy.calibrate.util import get_pointing_table
from scipy import ndimage
import os, sys
import copy
from PIL import Image
import datetime
import math
import pandas as pd
  
import cv2

%run ./functions.ipynb

In [2]:
############################################################################# 
# Apply sunpy mask to the BD image
def masking(map_img, segmented_img):
    # Binary image
    # when saving the binary images, the value =255 (useful for colours) corresponds to white (boolean value True, 1)
    # whereas the value =0 is black (False)
    # Note: some pixel are =150 : boundary pixels

    # Mask
    # boolean 1: pixel to be masked
    # boolean 0: not to be masked
    
    mask= segmented_img.data
    mask[segmented_img.data==0]=1 # to mask
    mask[segmented_img.data!=1]=0 # no mask
#     plt.imshow(mask)
    masked_img=sunpy.map.Map(map_img.data, map_img.meta, mask=mask)
    return masked_img


### Event choice

In [ ]:
# #             JUNE 14 2012
# event= 'E:/DATA/POLIMI/THESIS/.fits_files/Final-3_days/Event-2012-06-14'
# event_time='June 14 2012'
# flare_time='2012-06-14 14:35:00.630'

#             MARCH 07 2012
event= 'E:/DATA/POLIMI/THESIS/.fits_files/Final-3_days/Event-2012-03-07'
event_time='March 07 2012'
# flare_time='2012-03-07 00:42:00.630'
flare_time='2012-03-07 00:24:00'

# #             MARCH 08 2019
# event= 'E:/DATA/POLIMI/THESIS/.fits_files/Final-3_days/Event-2019-03-08'
# event_time='March 08 2019'
# flare_time='2019-03-08 03:37:00.630'

# ##       SEPTEMBER 6 2011
# event= 'E:/DATA/POLIMI/THESIS/.fits_files/Final-3_days/Event-2011-09-06'
# event_time='September 06 2011'
# flare_time='2011-09-06 22:22:00.630'
# peak= pd.to_datetime('2011-09-06T22:22:00.630')

## Compute brightness of each dimming image and plot evolution in time

- Load the cropped BD images and the cropped segmented images
- for each BD image apply the corresponding segmented image as a mask -> Python function: masking()
- Compute total brightness of the dimming surface for each map
- Plot the brightness evolution in time

In [ ]:
import sunpy.timeseries as ts
import pandas as pd
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter

# Load all the images
BD_img=sunpy.map.Map(event+'/BD/BD crop/*.fits', sequence=True)
seg_img=sunpy.map.Map(event+'/Segmented_Binary/*.fits', sequence=True)
print('Length of BD images: ', len(BD_img))
# time=ts.TimeSeries([BD_img[0]])
# Let us create a mask
# time= sunpy.timeseries.TimeSeries(BD_img[:], source='AIA_2')
# Binary image
# when saving the binary images, the value =255 (useful for colours) corresponds to white (boolean value True, 1)
# whereas the value =0 is black (False)

# Mask
# boolean 1: pixel to be masked
# boolean 0: not to be masked

masked_images=[]
dimming_int=[]
total_int=[]
time_string=[]

# print(BD_img[1].data)

for i in range(len(BD_img)):
# for i in range(1,2):
    masked_img = masking(BD_img[i], seg_img[i])
    mask=masked_img.mask
#     print(BD_img[i].data)
#     plt.figure()
#     masked_img.plot(vmin=-7, vmax=7)
#     print('Mask:\n', mask)
#     print('Map values where mask = 0:\n', masked_img.data[mask==0])
#     print('Number of non-masked pixels: ', len(masked_img.data[mask==0]))
    map_int=np.sum(np.array(masked_img.data[mask==0]))
#     print('Dimming total intensity: ', map_int)
    tot_int= np.sum(np.array(masked_img.data[~np.isnan(masked_img.data)]))
#     print('Map total intensity: ', tot_int, '\n')
    dimming_int.append(map_int)
    total_int.append(tot_int)
    
#     map_time=sunpy.time.parse_time(BD_img[i].meta['date-obs'])
#     map_time_string=(map_time.strftime("%Y%m%d-%H-%M-%S"))
    map_time_string=BD_img[i].meta['date-obs']
    time_string.append(map_time_string)

# ## 1st try
# print(time_string)
# # dimming_data= {'Date':time_string,'Dimming intensity':dimming_int}
# # dim_int_pd= pd.DataFrame(dimming_data)
# ts_dimming=pd.Series(time_string)
# ts_datetime=pd.to_datetime(ts_dimming)
# # print(ts_dimming)
# dim_int_pd= pd.DataFrame(dimming_int, index=ts_dimming)
# # print(dim_int_pd)
# # tot_int_data= {'Date':time_string,'Total intensity':total_int}
# # tot_int_pd= pd.DataFrame(tot_int_data)
# tot_int_pd= pd.DataFrame(total_int, index=ts_dimming)
# # print(tot_int_pd)


## 2 try
dim_int_pd= pd.DataFrame(data= {'Time': time_string, 'dimming_int': dimming_int, 'total_int': total_int})
# dim_int_pd["Time"] = pd.to_datetime(dim_int_pd["Time"])
print(dim_int_pd)
# tot_int_pd= pd.DataFrame(data= {'Time': time_string, })
# tot_int_pd["Time"] = pd.to_datetime(tot_int_pd["Time"])

dim_int_pd.to_csv(event+'/dimming_data.csv')

# time= ts.TimeSeries(time_string, concatenate=True)

# plt.rcParams["figure.figsize"] = [7.50, 3.50]
# plt.rcParams["figure.autolayout"] = True
# plt.plot(time, dimming_int, 'b', label='Dimming intensity')
# plt.plot(time, total_int, 'r', label='Total intensity')
# ax1= dim_int_pd.plot(color='blue', label='Dimming intensity')
# tot_int_pd.plot(ax=ax1, color='red', secondary_y=True, label='Total intensity')
# plt.plot(dim_int_pd)

#     intensity.append(map_int)
#     time_map=parse_time(BD_img[i].meta["date-obs"])
#     time_string=time_map.strftime("%Y%m%d-%H-%M-%S")
#     folder='D:/POLIMI/THESIS/.fits_files/2011_09_06-211A-dimming_validation/Masked dimming/'
#     write_file(folder+'masked_dimming-{string}.fits'.format(string=time_string),masked_img.data, masked_img.meta,'fits')       

In [7]:
df1= pd.read_csv(event+'/dimming_data.csv', index_col=0, parse_dates=True)
df1["Time"] = pd.to_datetime(df1["Time"] )
df1_plot=df1.copy()

df1_plot['dimming_int'] = df1_plot['dimming_int']/10e5
df1_plot['total_int'] = df1_plot['total_int']/10e5

fig = go.Figure()

fig.add_trace(go.Scatter(x=df1_plot['Time'], y=df1_plot['dimming_int'],mode='lines', line = dict(color='#636EFA'),legendgroup="brightness",name='dimming'))
fig.add_trace(go.Scatter(x=df1_plot['Time'], y=df1_plot['total_int'],mode='lines', line = dict(color='#EF553B'),legendgroup="brightness",name='total'))

fig.update_layout(title='Intensity over time (cropped image)<br>'+event_time,title_x=0.5,
                  title_font=dict(size=20, family='Arial'),
                  width=1700, height=700,template="plotly_white")

fig.update_yaxes(title_text="Brightness [10<sup>5</sup> DN]",mirror=True, ticks='inside', zeroline=False,
                 showline=True,linewidth=2,title_font=dict(size=21, family='Arial'),
                 color='black',tickwidth=2,tickfont=dict(size=18),linecolor='black', 
                 showgrid=False)

fig.update_xaxes(nticks=8,title_text=r"Time (UT)",mirror=True,
ticks='inside', #tickformat="%H:%M",
showline=True,linewidth=2,title_font=dict(size=20, family='Arial'),color='black',tickwidth=2,tickfont=dict(size=16),
                   linecolor='black',showgrid=False)
                 #,range=['2011-06-21 01:41','2011-06-21 05:00'])
    

fig.add_vline(x=flare_time, line_dash="dash", line_width=1, line_color="black")
if flare_time=='2011-09-06 22:22:00.630':
    fig.add_vline(x='2011-09-07 22:38:00.630', line_dash="dash", line_width=1, line_color="black")
    fig.add_vline(x='2011-09-08 15:36:00.630', line_dash="dash", line_width=1, line_color="black")
# fig.add_vline(x='2012-06-14 15:36:00.630', line_dash="dash", line_width=1, line_color="black")
# fig.add_hline(y='-0.4')

fig.show()
# fig.write_image(event+'/images/Brightness_evolution.png')